In [3]:
# import the necessary packages
from imutils import face_utils
import numpy as np
import imutils
import dlib
import cv2

def show_raw_detection(image, detector, predictor):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 그레이스케일 영상에서 면 검출
    rects = detector(gray, 1)
    # 얼굴 탐지를 반복
    for (i, rect) in enumerate(rects):
        # 얼굴 부위의 얼굴 랜드마크 결정, 얼굴 랜드마크(x, y) 좌표를 NumPy 배열로 변환
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        # dlib의 사각형을 OpenCV 스타일 경계 상자로 변환한 다음(x, y, w, h) 면 경계 상자를 그림
        (x, y, w, h) = face_utils.rect_to_bb(rect)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        # 얼굴 번호를 보여줌
        cv2.putText(image, "Face #{}".format(i + 1), (x - 10, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        # (x, y)-좌표를 통해 얼굴 랜드마크에 루프 적용 후 이미지 위에 그림
        for (x, y) in shape:
            cv2.circle(image, (x, y), 1, (0, 0, 255), -1)
    # 얼굴 감지 + 얼굴 랜드마크로 출력 이미지를 보여줌
    cv2.imshow("Output", image)
    cv2.waitKey(0)
def draw_individual_detections(image, detector, predictor):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 그레이스케일 영상에서 면 검출
    rects = detector(gray, 1)
    # 얼굴 탐지를 반복
    for (i, rect) in enumerate(rects):
        # 얼굴 부위의 얼굴 랜드마크 결정, 얼굴 랜드마크(x, y) 좌표를 NumPy 배열로 변환
        shape = predictor(gray, rect)
        shape = face_utils.shape_to_np(shape)
        # 얼굴 부분을 개별적으로 루프
        for (name, (i, j)) in face_utils.FACIAL_LANDMARKS_IDXS.items():
            # 원본 이미지를 복제해서 그 위에 그린 다음 얼굴 부분의 이름을 이미지에 표시
            clone = image.copy()
            cv2.putText(clone, name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            # 얼굴 랜드마크의 서브셋 위로 루프하여 특정 얼굴 부분을 그림
            for (x, y) in shape[i:j]:
                cv2.circle(clone, (x, y), 1, (0, 0, 255), -1)
                # 얼굴 부위의 ROI를 별도의 영상으로 추출
                (x, y, w, h) = cv2.boundingRect(np.array([shape[i:j]]))
                roi = image[y:y + h, x:x + w]
                roi = imutils.resize(roi, width=250, inter=cv2.INTER_CUBIC)
                # 특정 얼굴 부분을 보여줌
                cv2.imshow("ROI", roi)
                cv2.imshow("Image", clone)
                cv2.waitKey(0)
                # 투명한 오버레이로 모든 얼굴 표지를 시각화
                output = face_utils.visualize_facial_landmarks(image, shape)
                cv2.imshow("Image", output)
                cv2.waitKey(0)
                
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
# 입력 이미지를 로드하고 크기를 조정한 후 그레이 스케일로 변환
image = cv2.imread('image/chuu.jpg') 
image = imutils.resize(image, width=500)
# show_raw_detection(image, detector, predictor)
draw_individual_detections(image, detector, predictor)

KeyboardInterrupt: 